# A brief summary of *Cryptography*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents


## *Hashing* Algorithms

A *hashing* algorithm (or, function) maps any number of input bytes to a fixed number of output bytes, called the *hash*, or *digest*.

Hence, by definition, any hashing function is *lossy*.

<img src="../../.img/func-2in-1out.png" width="300" />

A hashing function is meant to posses the following characteristics:
* *Pre-image* resistance
    * Given a hash, it must be practically impossible, without brute-force, to find an input that maps to this hash.
* *Second Pre-Image* resistance
    * Given an input and its corresponding hash, it must be practically impossible, without brute-force, to find another input that maps to this hash.
* *Collision* resistance
    * It must be practically impossible, without brute-force, to find any two inputs that map to the same hash.

*Note:* Using brute-force, a *(Second) Pre-image* attack is of $O(2^n)$ time complexity, while a *Collision* attack is of $O(\sqrt{2^n})$ time complexity, where $n$ is the number of bits in the hash.

Historically, *MD-5* was used, until it was proven not to be *Collision* resistance.

Currently, the recommended hashing function is *SHA-256*.

| *Algorithm* | *Hash Size (bytes)* |
| --- | --- |
| *MD-5* | 16 |
| *SHA-256* | 32 |

In [3]:
import hashlib
sha256_hasher = hashlib.sha256(b'Hello, world.')
sha256_hasher.hexdigest()

'f8c3bf62a9aa3e6fc1619c250e48abe7519373d3edf41be62eb5dc45199af2ef'

In [9]:
import hashlib
sha256_hasher = hashlib.sha256()
sha256_hasher.update(b'Hello,')
sha256_hasher.update(b' world.')
sha256_hasher.hexdigest()

'f8c3bf62a9aa3e6fc1619c250e48abe7519373d3edf41be62eb5dc45199af2ef'

## *Encryption*

*Encryption* transforms so-called *plain-text*, into incomprehensible *cipher-text*.

Encryption algorithms are divided into two categorites:
* *Block Cipher* algorithms, operate on blocks (e.g: 16 bytes per block).
* *Stream Cipher* algorithms, operate on bytes.

For block cipher algorithms, when the data length is not *block-aligned*, padding is added. Padding schemes include,

| *Scheme* | `CASE`: 3 bytes of padding are required | `CASE`: 2 bytes of padding are required |
| --- | --- | --- |
| *PKCS7* | `0x03 0x03 0x03` | `0x02 0x02` |
| *ANSI X.923* | `0x00 0x00 0x03` | `0x00 0x02` |

### *Symmetric* Encryption

*Symmetric* encryption algorithms use the same *key* to encrypt and decrypt. The cipher-text is deterministic, and, ideally, unique per key.

*AES*, a 16-byte block cipher algorithm, is, currently, the recommended symmetric encryption algorithm.

It has many modes of operation, including,
* *Electronic Code Book (ECB)*,
* *Cipher Block Chaining (CBC)*, and,
* *Counter (CTR)*.

*Note:* *AES* supports different key sizes, including *128*, *192*, and *256* bits.

#### *AES: ECB* mode

In *ECB* mode, each block is encrypted *independent* of any other block.

In [46]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

def test_cipher(cipher, key, data):
    
    # ACTION: Instantiate useful objects.
    encryptor = cipher.encryptor()
    decryptor = cipher.decryptor()
    padder = padding.PKCS7(128).padder()
    unpadder = padding.PKCS7(128).unpadder()

    cipher_text = []
    decrypted_text = []

    # ACTION: Encrypt.
    for item in data:
        str_2_bin = item.encode('ascii')
        cipher_text.append(encryptor.update(padder.update(str_2_bin)))
    cipher_text.append(encryptor.update(padder.finalize()))
    cipher_text_hex = [item.hex() for item in cipher_text]

    # ACTION: Decrypt.
    for item in cipher_text:
        decrypted_text.append(unpadder.update(decryptor.update(item)))
    decrypted_text.append(unpadder.finalize())
    decrypted_text_hex = [item.hex() for item in decrypted_text]

    # ACTION: Print summary.
    data_concat = ''.join(data)
    cipher_text_concat = ''.join(cipher_text_hex)
    decrypted_text_concat = ''.join(decrypted_text_hex)
    decrypted_text_ascii = b''.join(decrypted_text).decode('ascii')

    print('Key Size (Bits)               ->    ', key_size*8)
    print('Key                           ->    ', key.hex())
    print('Plain Text (ASCII)            ->    ', data_concat)
    print('Plain Text Length (Bytes)     ->    ', len(data_concat))
    print('Cipher Text (HEX)             ->    ', cipher_text_concat)
    print('Cipher Text Length (Bytes)    ->    ', len(cipher_text_concat)//2)
    print('Decrypted Text (HEX)          ->    ', decrypted_text_concat)
    print('Decrypted Text (ASCII)        ->    ', decrypted_text_ascii)
    print('Decrypted Text Length (Bytes) ->    ', len(decrypted_text_concat)//2)


In [49]:
# Dummy sentence.
data = ["I ", "am very ", "hungry!"]

# Random key, 128 bits in length.
key_128 = os.urandom(16)

# AES, ECB mode.
aes_cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())

# ...
test_cipher(cipher=aes_cipher, key=key_128, data=data)

Key Size (Bits)               ->     128
Key                           ->     2ba1279c0c5033104aa841806dd392d8
Plain Text (ASCII)            ->     I am very hungry!
Plain Text Length (Bytes)     ->     17
Cipher Text (HEX)             ->     8cc6047cb18104c8dd045dfefda942928e0456f5bf729e52987818825dbacb2b
Cipher Text Length (Bytes)    ->     32
Decrypted Text (HEX)          ->     4920616d20766572792068756e67727921
Decrypted Text (ASCII)        ->     I am very hungry!
Decrypted Text Length (Bytes) ->     17


#### *AES: CBC* mode 